In [1]:
from cloudvolume import CloudVolume, Skeleton

/Users/thomasathey/Documents/mimlab/mouselight/env/lib/python3.8/site-packages/python_jsonschema_objects/__init__.py:50: UserWarning: Schema version http://json-schema.org/draft-04/schema not recognized. Some keywords and features may not be supported.
  warnings.warn(


In [2]:
dest_segments = "s3://open-neurodata/brainlit/brain2_segments_solange"
vol = CloudVolume(dest_segments)

In [3]:
with open("/Users/thomasathey/Documents/mimlab/mouselight/solange/Neuron_1_axon_consensus_final.swc") as f:
    text = f.read()
skel = Skeleton.from_swc(text)

In [8]:
type(skel.vertices.tolist()[0][0])

float

In [22]:
import struct
from io import BytesIO
# assume you have a skeleton object already
# called `skel`
skel_bio = BytesIO()
# write the number of vertices and edges
skel_bio.write(struct.pack('<I',skel.vertices.shape[0]))
skel_bio.write(struct.pack('<I',skel.edges.shape[0]))
# write the actual vertices and edges
skel_bio.write(skel.vertices.astype('float32').tostring())
skel_bio.write(skel.edges.astype('uint32').tostring())
# write the file
with open('/Users/thomasathey/Documents/mimlab/mouselight/solange/101', 'wb') as fp:
    fp.write(skel_bio.getbuffer())

In [19]:
skel.edges

array([[   1,    0],
       [   2,    1],
       [   3,    2],
       ...,
       [2623, 2622],
       [2624, 2623],
       [2625, 2619]], dtype=uint32)

In [131]:
print(skel)
print(skel2)

Skeleton(segid=101, vertices=(shape=3783, float64), edges=(shape=3782, uint32), radius=(3783, float32), vertex_types=(3783, uint8), space='physical' transform=[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0]])
Skeleton(segid=2, vertices=(shape=7872, float32), edges=(shape=7871, uint32), radius=(7872, float32), vertex_types=(7872, uint8), space='physical' transform=[[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0]])


In [130]:
skel.id = 101
skel.space= 'physical'

In [100]:
import numpy as np
skel.vertices = (skel.vertices + np.array([[62538.535211, 52094.467848, 24439.070841]]))

In [128]:
skel.vertices = ((skel.vertices - np.amin(skel.vertices,axis=0))*100 + np.array([[ 785976.4,  1485262.6,   468177.62]]))

In [137]:
print(np.amin(skel2.vertices,axis=0))
print(np.amax(skel2.vertices,axis=0))
print(np.amin(skel.vertices,axis=0))
print(np.amax(skel.vertices,axis=0))

[ 785976.4  1485262.6   468177.62]
[9928148.  5241548.  5606111.5]
[ 785976.4  1485262.6   468177.62]
[1005099.196875 1951454.19375  1319034.87    ]


In [147]:
vol.skeleton.upload(skel2)

Uploading: 100%|██████████| 1/1 [00:00<00:00,  2.52it/s]


In [149]:
vol.skeleton.upload_raw(2, skel2.vertices, skel2.edges, skel2.radii, skel2.vertex_types)

Uploading: 100%|██████████| 1/1 [00:00<00:00,  3.05it/s]


Uploading: 100%|██████████| 1/1 [00:00<00:00,  3.94it/s]


In [86]:
skel2.vertices

array([[8576797. , 2549391.5, 2890955. ],
       [8575797. , 2550399.5, 2890955. ],
       [8547766. , 2563266.5, 2905509.8],
       ...,
       [8537523. , 2426778.2, 2885632.8],
       [8541164. , 2414137.8, 2878021.5],
       [8530156. , 2411641.5, 2879613.2]], dtype=float32)

# Download then Upload

In [16]:
og_path = "s3://open-neurodata/brainlit/brain2_segments"
vol = CloudVolume(og_path)
skel = vol.skeleton.get(1)

Downloading: 100%|██████████| 1/1 [00:00<00:00, 12.07it/s]


In [7]:
skel101 = skel
skel101.id = 101

In [8]:
vol.skeleton.upload(skel101)

Uploading: 100%|██████████| 1/1 [00:00<00:00,  3.64it/s]


In [9]:
Skeleton.equivalent(skel, skel101)

True

In [144]:
skel2 == skel101

True

In [143]:
skel101.id = 2